In [41]:
from 有用なスクリプト import load_txt
for sentence in load_txt(path="./吾輩は猫であるutf8.txt")[:10]:
    if len(sentence) > 1:
        print(sentence if len(sentence) < 50 else sentence[:50]+"　（以下略）")

吾輩は猫である

夏目漱石

　吾輩《わがはい》は猫である。名前はまだ無い。

　どこで生れたかとんと見当《けんとう》がつかぬ。何でも薄暗いじめじめした所でニャーニャー泣いていた事　（以下略）
　この書生の掌の裏《うち》でしばらくはよい心持に坐っておったが、しばらくすると非常な速力で運転し始め　（以下略）
　ふと気が付いて見ると書生はいない。たくさんおった兄弟が一｜疋《ぴき》も見えぬ。肝心《かんじん》の母　（以下略）
　ようやくの思いで笹原を這い出すと向うに大きな池がある。吾輩は池の前に坐ってどうしたらよかろうと考え　（以下略）
　吾輩の主人は滅多《めった》に吾輩と顔を合せる事がない。職業は教師だそうだ。学校から帰ると終日書斎に　（以下略）
　吾輩がこの家へ住み込んだ当時は、主人以外のものにははなはだ不人望であった。どこへ行っても跳《は》ね　（以下略）


In [42]:
from 有用なスクリプト import train_tokenizer

tokenizer = train_tokenizer(
    file_path="./吾輩は猫であるutf8.txt",   # テキスト元になるファイル
    vocab_size=128,                       # 辞書の大きさ
    text_to_compress = "吾輩わがはいは猫である。名前はまだ無い。"
)

Tokenizerの指定単語数は 128...
訓練完了

--- テキスト容量比較 ---
比較用サンプル: '吾輩わがはいは猫である。名前はまだ無い。'

[UTF-8 容量]
  bytes: 60
  bits (bytes * 8): 480

[BPE 容量]
  辞書容量: 128
  トークンごとのbits密度 (ceil(log2(vocab_size))): 7
  トークン数: 60
  トークン: [85, 110, 78, 88, 76, 58, 83, 96, 109, 83, 95, 106, 83, 95, 63, 83, 95, 98, 83, 95, 63, 87, 106, 60, 83, 95, 56, 83, 95, 96, 83, 96, 105, 83, 94, 96, 85, 110, 107, 85, 103, 107, 83, 95, 63, 83, 95, 78, 83, 95, 126, 87, 98, 50, 83, 95, 98, 83, 94, 96]
  bits (トークン数 * bits_per_token): 420

--- 📊 最終結果 ---
UTF-8 Bits: 480
BPE Bits:   420
-> 圧縮比率: 1.14 (UTF-8 bits / BPE bits)


In [43]:
from 有用なスクリプト import load_txt

for sentence in load_txt(path="./吾輩は猫であるutf8.txt"):
    assert tokenizer.decode(tokenizer.encode(sentence).ids) == sentence
        

In [44]:
適当な文章 = "論文を書くのは辛い El Psy Congroo 🥺"
print("適当な文章:\t", 適当な文章)
トークン = tokenizer.encode(適当な文章).ids
print("トークン: \t", トークン[:15])
日本語文書 = tokenizer.decode(トークン)
print("復元した日本語:\t", 日本語文書)


適当な文章:	 論文を書くのは辛い El Psy Congroo 🥺
トークン: 	 [88, 60, 116, 86, 116, 101, 83, 96, 112, 86, 121, 72, 83, 95, 109]
復元した日本語:	 論文を書くのは辛い l sy ongroo ���


In [45]:

tokenizer = train_tokenizer(
    file_path=[         # テキスト元になるファイル
        "./吾輩は猫であるutf8.txt",
        "./text/emoji.txt",
        "./text/harry potter 1.txt"
    ],                  
    vocab_size=5000,    # 辞書の大きさ
    text_to_compress = "吾輩わがはいは猫である。名前はまだ無い。"
)
print("----------------------------")
適当な文章 = "月がきれいです El Psy Congroo 🥺"
print("適当な文章:\t", 適当な文章)
トークン = tokenizer.encode(適当な文章).ids
print("トークン: \t", トークン[:15])
日本語文書 = tokenizer.decode(トークン)
print("復元した日本語:\t", 日本語文書)


Tokenizerの指定単語数は 5000...
訓練完了

--- テキスト容量比較 ---
比較用サンプル: '吾輩わがはいは猫である。名前はまだ無い。'

[UTF-8 容量]
  bytes: 60
  bits (bytes * 8): 480

[BPE 容量]
  辞書容量: 5000
  トークンごとのbits密度 (ceil(log2(vocab_size))): 13
  トークン数: 13
  トークン: [460, 3696, 813, 167, 808, 324, 164, 2836, 167, 1158, 566, 151, 164]
  bits (トークン数 * bits_per_token): 169

--- 📊 最終結果 ---
UTF-8 Bits: 480
BPE Bits:   169
-> 圧縮比率: 2.84 (UTF-8 bits / BPE bits)
----------------------------
適当な文章:	 月がきれいです El Psy Congroo 🥺
トークン: 	 [493, 170, 230, 199, 151, 314, 112, 26, 51, 112, 32, 58, 64, 300, 46]
復元した日本語:	 月がきれいです El Psy ongroo 🥺


In [46]:
パターン１ = [1, 2, 3, 4, 5, ...]
パターン２ = [1, 4, 9, 16, 25, ...]
パターン３ = [1, 2, 4, 8, 16, ...]

In [47]:
def パターン１(x: int):
    # ｘが１から５の時、ｘしか値として戻っていない；
    return x + (x - 1) * (x - 2) * (x - 3) * (x - 4) * (x - 5) 


新パターン１ = [パターン1(x) for x in range(1, 7)]
print("新パターン１ ->", 新パターン1)

新パターン１ -> [1, 2, 3, 4, 5, 126]


In [ ]:
文書 = "月がきれい"
for idx in range(0, len(文書)):
    文書断片 = 文書[: idx + 1]
    tokens = tokenizer.encode(文書断片)
    print("　" * (len(文書) - idx), 文書断片, ":", end=" ")
    print(tokens.ids)

　　　　　 月 : [493]
　　　　 月が : [493, 170]
　　　 月がき : [493, 170, 230]
　　 月がきれ : [493, 170, 230, 199]
　 月がきれい : [493, 170, 230, 199, 151]


In [ ]:
import torch
from transformers import AutoTokenizer, Gemma3ForCausalLM

ckpt = "google/gemma-3-1b-pt"
tokenizer = AutoTokenizer.from_pretrained(ckpt)
model = Gemma3ForCausalLM.from_pretrained(
    ckpt,
    torch_dtype=torch.bfloat16,
    device_map="auto"
)




`torch_dtype` is deprecated! Use `dtype` instead!
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/2.00G [00:00<?, ?B/s]

c:\Users\xhaug\Playground\ＣＭ１０７\Easy_LLM_Watermark\.venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\xhaug\.cache\huggingface\hub\models--google--gemma-3-1b-pt. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


generation_config.json:   0%|          | 0.00/215 [00:00<?, ?B/s]

The following generation flags are not valid and may be ignored: ['top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


 the heart of Paris, France.The Eiffel Tower is a symbol of Paris and France.The Eiffel Tower is a symbol of Paris and France.The Eiffel Tower is a symbol of Paris and France.The Eiffel Tower is a symbol


In [9]:
prompt = "東京は日本の"
model_inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

input_len = model_inputs["input_ids"].shape[-1]

with torch.inference_mode():
    generation = model.generate(**model_inputs, max_new_tokens=1, do_sample=False)
    generation = generation[0][input_len:]

decoded = tokenizer.decode(generation, skip_special_tokens=True)
print(decoded)

首都


In [ ]:
prompt = "東京は日本の"
max_new_token = 13
for i in range(max_new_token):
    model_inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    input_len = model_inputs["input_ids"].shape[-1]

    with torch.inference_mode():
        generation = model.generate(**model_inputs, max_new_tokens=1, do_sample=False)
        generation = generation[0][input_len:]

    decoded = tokenizer.decode(generation, skip_special_tokens=True)
    print(prompt+decoded)
    prompt +=decoded

IndentationError: unexpected indent (4242513369.py, line 1)